# Pré-Processamento dos Dados

## Título:

**Predição de preço de imóveis**

## Membros:

*   Adrisson Rogério Samersla
*   Nickolas Batista Mendonça Machado
*   Thayna Pires Baldão



# Setup

In [ ]:
# Importando os pacotes necessários para a análise

import os

import pandas            as pd
import geopandas         as gpd
import numpy             as np
import scipy             as sc
import matplotlib.pyplot as plt
import seaborn           as sns

from google_drive_downloader import GoogleDriveDownloader as gdd

import mapclassify

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.reset_option('max_colwidth')

In [ ]:
# Baixando o dataset

dataset_dir = "../dataset"
has_dataset_dir = os.path.isdir(dataset_dir)
if (not has_dataset_dir):
  # Link of dataset folder: 
  # https://drive.google.com/file/d/1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs/view?usp=sharing
  gdd.download_file_from_google_drive(file_id='1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs',
                                      dest_path='../dataset.zip',
                                      showsize=True,
                                      unzip=True)

In [ ]:
# Lendo a base de dados

df = pd.read_csv(dataset_dir + '/dataset.csv')
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))

In [ ]:
df.dtypes

In [ ]:
df.head()

# Seleção Manual de Atributos

Atributos a serem eliminados:

* id 

In [ ]:
ignore_columns = [
    'id',
    'property_id',
    'operation',
    'place_name',
    'place_with_parent_names',
    'country_name',
    'state_name',
    'geonames_id',
    'lat_lon',
    'currency',
    'floor',
    'description',
    'title',
    'image_thumbnail',
    'collected_on'
]

def manual_selection(dataset):
    return dataset.drop(columns=ignore_columns)

# filtered = manual_selection(df)
# for col in ignore_columns:
#     assert col not in manual_selection(df).columns

# Limpeza de dados

## Eliminando Inconsistências

- Moedas Estrangeiras

In [ ]:
df.currency.astype('str').value_counts()

In [ ]:
def filter_currency(dataset):
    positions = dataset['currency'] == 'BRL'
    return dataset[positions]

# filter_currency(df).currency.astype('str').value_counts()

- Objetos Fora do Brasil

In [ ]:
df.state_name.astype('str').value_counts()

In [ ]:
def filter_latlon(dataset):
    data = dataset[['lat', 'lon']]

    # data = data.dropna()
    # points = gpd.points_from_xy(data.lon, data.lat, crs=map_df.crs)
    # points = gpd.GeoSeries(points)
    # positions_in = map_df.contains(points)

    positions = data.lat.notna().squeeze() & data.lon.notna().squeeze()

    lon_max = -30.0
    lon_min = -80.0
    lat_min = -40.0
    lat_max = 10.0
    return dataset.loc[
        positions & 
        (dataset.state_name != "Outros países") &
        ((data.lat > lat_min) & (data.lat < lat_max)) & 
        ((data.lon > lon_min) & (data.lon < lon_max))
    ]

# filter_latlon(df).state_name.astype('str').value_counts()

## Lidando com dados incompletos

- `surface_total_in_m2` e `surface_covered_in_m2`

In [ ]:
def fill_surface_total_with_surface_covered(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_covered_in_m2'], inplace=True)

def fill_surface_covered_with_surface_total(dataset):
    dataset['surface_covered_in_m2'].fillna(dataset['surface_total_in_m2'], inplace=True)

def fill_remaining_objects_without_surfaces(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_covered_in_m2'].mean(), inplace=True)
    dataset['surface_covered_in_m2'].fillna(dataset['surface_total_in_m2'].mean(), inplace=True)

def drop_remaining_objects_without_surfaces(dataset):
    dataset.dropna(subset=['surface_total_in_m2'], how='all', inplace=True)

fill_surface_total_with_surface_covered(df)
fill_surface_covered_with_surface_total(df)
drop_remaining_objects_without_surfaces(df)
# df.isnull().sum()*100/1_000_000

- `expenses`

In [ ]:
def fill_expenses(dataset):
    dataset['expenses'].fillna(0.0, inplace=True)

# fill_expenses(df)
# df.isnull().sum()*100/1_000_000

- `rooms`

In [ ]:
def fill_rooms(dataset):
    dataset['surface_category'] = pd.qcut(dataset['surface_covered_in_m2'],4)
    dataset['rooms'] = dataset['rooms'].fillna(dataset.groupby('surface_category')['rooms'].transform('mean').round())

# fill_rooms(df)
# df.isnull().sum()*100/1_000_000

# Aplicando o Pré-Processamento

In [ ]:
df = filter_currency(df)
df = filter_latlon(df)

df = manual_selection(df)
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))
print()

df.dtypes

In [ ]:
df.to_csv(dataset_dir + "/preprocessed.csv", index=False)